# 3.Data Preparation of Lyrics I

Tasks to perform:
1. Text pre-processing of Lyrics

In [25]:
# import modules
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import spacy as sc
import unicodedata
import contractions

In [26]:
# read data
songs_df = pd.read_csv('Data/spotify_sg_complete_3.csv')
songs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1662 entries, 0 to 1661
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   song_id             1662 non-null   object 
 1   popularity          1662 non-null   float64
 2   title               1662 non-null   object 
 3   artist              1662 non-null   object 
 4   album/single        1662 non-null   object 
 5   artist_followers    1662 non-null   int64  
 6   explicit            1662 non-null   int64  
 7   album               1662 non-null   object 
 8   release_date        1662 non-null   object 
 9   danceability        1662 non-null   float64
 10  energy              1662 non-null   float64
 11  key                 1662 non-null   int64  
 12  loudness            1662 non-null   float64
 13  mode                1662 non-null   int64  
 14  speechiness         1662 non-null   float64
 15  acousticness        1662 non-null   float64
 16  instru

In [27]:
# remove copyright str

def remove_copyright(lyrics, genre):
    clean_list = lyrics.split('\n')[:-4]
    if len(clean_list) <5:
        clean_list = lyrics.split('  ')[:-4]
    clean_lyrics = ','.join(clean_list)
    return clean_lyrics

songs_df['lyrics'] = songs_df.apply(lambda t: remove_copyright(t.lyrics, t.genre), axis=1)

In [28]:
def strip_accents(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) 
    if unicodedata.category(c) != 'Mn')

In [29]:
# pre-processing

def preprocess_text(nlp, text, selected_tags=None):

    # remove punctuation except apostrophes
    text = re.sub(r'[^\w\d\s\']+', ' ', text)

    # expand contractions
    expanded_text = ' '.join([contractions.fix(word) for word in text.split()])

    # tokenize
    text = nlp(text)
    tokens = [(w.text, w.pos_) for w in text]
    
    # remove unwanted pos tags
    tokens_rm_pos = tokens
    if selected_tags is not None:
        tokens_rm_pos = [(w, pos) for w, pos in tokens if pos in selected_tags]
    
    # remove remaining punctuation and case normalize
    tokens_rm_pn = [(w.lower(), pos) for w, pos in tokens_rm_pos if w.encode().isalpha()]
      
    # remove accented characters
    tokens_unaccent = [(strip_accents(w), pos) for w, pos in tokens_rm_pn]

    # remove stop words
    stop_words = stopwords.words('english')
    # stop_words.extend([])
    stop_words = set(stop_words)
    tokens_rm_stop = [(w, pos) for w,pos in tokens_unaccent if not w in stop_words]

    # remove vocables
    patterns = [re.compile(r'a+h+'), re.compile(r'w+o+'), re.compile(r'u+h+'), re.compile(r'a+y+'), re.compile(r'p+a+'), 
            re.compile(r'l+a+'), re.compile(r'p+a+n'), re.compile(r'badda'), re.compile(r'bing'), re.compile(r'boom'),
            re.compile(r'n+a+'), re.compile(r'umpah'), re.compile(r'h+m+'), re.compile(r'e+h+'),re.compile(r'whoa'), 
            re.compile(r'woah'), re.compile(r'd+y+'),re.compile(r'd+e+'), re.compile(r'h+e+y+'), re.compile(r'h+a+'),
            re.compile(r'y+e+a+h'), re.compile(r'm+h+'), re.compile(r'y+a+'), re.compile(r'y+e+'), re.compile(r'y+u+h'),
            re.compile(r'ababwa'), re.compile(r'ab'), re.compile(r'aaaaaow'), re.compile(r'aa'), re.compile(r'zzann'), 
            re.compile(r'z+t+'), re.compile(r'thing'), re.compile(r'o+h+'), re.compile(r'd+o'), re.compile(r'd+u+m'), 
            re.compile(r'q+u+e'), re.compile(r'e'), re.compile(r'ai'), re.compile('[a-z]a')]

    tokens_rm_voc = [(w, pos) for w, pos in tokens_rm_stop if not any(pattern.match(w) for pattern in patterns)]

    # remove 2 letter words
    tokens_rm_fin =[(w, pos) for w, pos in tokens_rm_voc if len(w) > 2] 
    
    # lemmatizaton
    wnl = WordNetLemmatizer()
    tokens_lemmed = [wnl.lemmatize(w) for w, pos in tokens_rm_fin]
    
    # convert to str
    cleaned = ', '.join(tokens_lemmed)

    return cleaned

In [30]:
# test all tokens with one song
!pip install -U spacy --user
!python -m spacy download en_core_web_sm

nlp = sc.load('en_core_web_sm')
songs_df_sample = songs_df.sample()
songs_df_sample['tokens_all'] = songs_df_sample['lyrics'].apply(lambda t: preprocess_text(nlp, t))
print('cleaned:' ,songs_df_sample.tokens_all.item())
print('')
print('og:', songs_df_sample.lyrics.item())

  Using cached spacy-3.2.0-cp38-cp38-win_amd64.whl (12.1 MB)
  Using cached thinc-8.0.13-cp38-cp38-win_amd64.whl (1.0 MB)
  Using cached spacy_loggers-1.0.1-py3-none-any.whl (7.0 kB)
  Using cached langcodes-3.2.1-py3-none-any.whl
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.5
    Uninstalling thinc-7.4.5:
      Successfully uninstalled thinc-7.4.5


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\User\\AppData\\Roaming\\Python\\Python38\\site-packages\\~-inc\\extra\\search.cp38-win_amd64.pyd'
Check the permissions.

You should consider upgrading via the 'C:\Users\User\.conda\envs\fyp-env2\python.exe -m pip install --upgrade pip' command.
Traceback (most recent call last):
  File "C:\Users\User\.conda\envs\fyp-env2\lib\runpy.py", line 185, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "C:\Users\User\.conda\envs\fyp-env2\lib\runpy.py", line 144, in _get_module_details
    return _get_module_details(pkg_main_name, error)
  File "C:\Users\User\.conda\envs\fyp-env2\lib\runpy.py", line 111, in _get_module_details
    __import__(pkg_name)
  File "C:\Users\User\AppData\Roaming\Python\Python38\site-packages\spacy\__init__.py", line 10, in <module>
    from thinc.neural.util import prefer_gpu, require_gpu
ModuleNotFoundError: No module named

KeyError: "[E002] Can't find factory for 'tok2vec'. This usually happens when spaCy calls `nlp.create_pipe` with a component name that's not built in - for example, when constructing the pipeline from a model's meta.json. If you're using a custom component, you can write to `Language.factories['tok2vec']` or remove it from the model meta and add it via `nlp.add_pipe` instead."

In [ ]:
# preprocess lyrics
nlp = sc.load('en_core_web_sm')
songs_df['tokens_all'] = songs_df['lyrics'].apply(lambda t: preprocess_text(nlp, t))
songs_df['tokens_nouns'] = songs_df['lyrics'].apply(lambda t: preprocess_text(nlp, t, selected_tags=('NOUN')))
songs_df['tokens_desc'] = songs_df['lyrics'].apply(lambda t: preprocess_text(nlp, t, selected_tags=('ADJ', 'ADV')))

In [ ]:
# remove songs with no meaningful tokens
final_songs_df = songs_df[songs_df.tokens_nouns.str.len() > 1]
final_songs_df.info()

(33, 29)

In [ ]:
# write to csv
final_songs_df.to_csv('Data/spotify_sg_tokens_4.csv', index=False)